# Проект: защита данных клиентов страховой компании 

**Цель**: Разработать такой метод преобразования персональных данных клиентов, чтобы по ним было сложно восстановить персональную информацию, но сохранилась возможность делать предсказания целевого признака без потери точности.

## 1. Изучение набора данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Открываем датафрейм
data = pd.read_csv('../../../insurance/datasets/insurance.csv')

Посмотрим, что из себя представляют данные.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
# Переименуем столбцы
data.columns=['is_male', 'age', 'salary', 'relatives', 'times_paid']

In [5]:
data.head()

,is_male,age,salary,relatives,times_paid
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [6]:
# Проверим дубликаты
data.duplicated().sum()

153

- Столбцы переведены на английский.
- Пропусков нет.
- Типы данных уже соответствуют признакам, которые они описывают. Не будем с ними ничего делать. Тем более, что модель при предсказании всё равно работает с `float`.
- Присутствуют дубликаты, но мы не будем их удалять, потому что у нас нет такой задачи.

Разделим данные на признаки и целевой признак и преобразуем в векторы NumPy.

In [7]:
X = data.drop('times_paid', axis=1).values
y = data.times_paid.values
X, y

(array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
        [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
        [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
        ...,
        [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
        [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
        [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]]),
 array([0, 1, 0, ..., 0, 0, 0]))

In [8]:
def print_shapes(*arrays):
    """Принимает список массивов и печатает их размеры."""
    for a in arrays:
        print(a.shape)

In [9]:
print_shapes(X, y)

(5000, 4)
(5000,)


---

## 2. Умножение матриц
Вопрос, на который мы будем искать ответ, будет звучать так: ***Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?***

### Качество линейной регрессии без изменения признаков
Прежде, чем выяснять, изменится ли качество линейной регрессии, мы зафиксируем качество регрессии на исходных данных без изменений.

In [10]:
class Linear_r2:
    """Обучает линейную регрессию на признаках, содержит поля 'R2' и 'модель'."""
    def __init__(self, features, target):
        self.model = LinearRegression()
        self.preds = self.model.fit(features, target).predict(features)
        self.score = r2_score(target, self.preds)

In [11]:
r2_initial = Linear_r2(X, y)
r2_initial.score

0.4249455028666801

По умолчанию регрессия даёт R2 == *0.42*. Теперь мы знаем, от чего отталкиваться.

---

### Качество линейной регрессии с трансформацией признаков
Попробуем создать произвольную обратимую матрицу **P** и умножить на неё наши признаки.

In [12]:
# Фиксируем случайное состояние
np.random.seed(0)

# Генерируем квадратную матрицу
P = np.random.randint(100, size=(X.shape[1], X.shape[1]))
P

array([[44, 47, 64, 67],
       [67,  9, 83, 21],
       [36, 87, 70, 88],
       [88, 12, 58, 65]])

In [13]:
# Проверим, обратима ли матрица P
P_inv = np.linalg.inv(P)
P_inv
# Да, обратима

array([[-0.08489421,  0.0109745 ,  0.0407572 ,  0.02878175],
       [-0.09272562,  0.0138436 ,  0.05851608,  0.0118844 ],
       [ 0.05834508,  0.00787096, -0.02912716, -0.02324955],
       [ 0.07999051, -0.02443685, -0.03999156, -0.00502974]])

In [14]:
# Преобразуем признаки через матрицу P
X_transformed = X @ P
X_transformed.shape

(5000, 4)

In [15]:
X_transformed[:5]

array([[1788479., 4315628., 3475525., 4365793.],
       [1371170., 3306426., 2663876., 3345031.],
       [ 757943., 1827261., 1472407., 1848609.],
       [1502783., 3628113., 2920859., 3670171.],
       [ 941520., 2270999., 1829388., 2297455.]])

Обучим линейную регрессию и измерим её R2.

In [16]:
r2_transformed = Linear_r2(X_transformed, y)
r2_transformed.score

0.42494550286668675

Значение R2 изменилось. Сильно ли? Сравним точности как `float`, держа в уме, что у этого типа данных ограниченная точность.

In [17]:
def float_equal(a, b, threshold=1e-6):
    """Проверяет равеноство двух float с использованием порога."""
    return np.abs(a - b) < threshold

In [18]:
float_equal(r2_transformed.score, r2_initial.score)

True

Как мы видим, **точность предсказаний не пострадала**.

Попробуем понять, почему предсказания не ухудшились, хотя значения признаков стали совершенно нечеловекопонятными. Поищем доказательство того, что умножение на обратимую матрицу не влияет на предсказание. Используем для этого формулу предсказаний линейной регрессии.

![img](math/math0.png)

In [19]:
# Веса модели, обученной на изменённых признаках
w_transformed = r2_transformed.model.coef_

# Веса модели, обученной на исходных признаках
w_initial = r2_initial.model.coef_

Умножим матрицу **P^-1** на вектор обычных весов и сравним с изменённым вектором весов.

In [20]:
# Векторы весов должны совпасть
all(float_equal(P_inv @ w_initial, w_transformed))

True

И да, они совпали, равенство выполняется. А теперь **самое интересное**:

Если формула изменённых весов содержит в себе матрицу P^-1, то при предсказании матрица признаков преобразуется обратно в исходную:

![img](math/math1.png)

Мы вернулись к первоначальной формуле.

### Что это всё означает?
- Исследуя механизм предсказаний линейной регрессии, мы выяснили, что матрица-преобразователь $P$ в итоге **выпадает из конечной формулы** предсказания, потому что умножается на обратную себе матрицу и становится единичной матрицей $E$.
- По такому же принципу мы могли *умножить все признаки на какой-нибудь случайный скаляр, а затем разделить на него же*. Но это упростило бы восстановление исходных значений.

И нашим ответом будет: ***если умножить признаки на обратимую матрицу, качество линейной регрессии не изменится.***

---

### \*Умножение признаков на необратимую матрицу
А что, если мы создадим **необратимую матрицу** и умножим признаки на неё? *Будет ли регрессия вообще работать?* Ведь если у нас получится, то мы зашифруем признаки так, что их уже не получится восстановить и похитить.

In [21]:
# Создадим матрицу из единиц
ones = np.ones((4, 4))
ones

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [22]:
# Выполним умножение
X_no_inv = X @ ones
X_no_inv.T

array([[49643., 38047., 21029., ..., 33922., 32726., 40630.],
       [49643., 38047., 21029., ..., 33922., 32726., 40630.],
       [49643., 38047., 21029., ..., 33922., 32726., 40630.],
       [49643., 38047., 21029., ..., 33922., 32726., 40630.]])

Пока что видим, что все столбцы матрицы признаков заполнились одинаковыми числами.

In [23]:
Linear_r2(X_no_inv, y).score

0.00020770518345514244

Как становится понятно из нашего эксперимента, умножение признаков на необратимую матрицу **ломает качество** модели и делает её совершенно бесполезной без возможности вернуть всё как было.

---

## 3. Алгоритм преобразования
Зная то, что мы узнали, мы можем написать класс, который будет:
- Создавать случайную матрицу-ключ и запоминать её внутри себя как **приватное поле** `__P`, чтобы **нельзя** было **получить** к нему **доступ извне**,
- **Шифровать и расшифровывать** матрицу признаков при помощи матрицы-ключа в стиле `sklearn`.

In [24]:
class RandomCipher:
    """Объект-шифровщик.
    Преобразует данные, умножая их на случайную обратимую матрицу. Умеет восстанавливать данные """
    
    def __init__(self, random_state=None):
        """random_state: фиксирует случайное состояние для случайной трансформации."""
        self.__rstate = random_state
        
    def fit(self, X):
        """Принимает матрицу и создаёт внутри себя матрицу-ключ.""" 
        self.__make_matrix(X)
        return self   
        
    def transform(self, X):
        """Преобразует матрицу ключом."""
        return X @ self.__P
    
    def inverse_transform(self, X):
        """Умножает матрицу на обратную матрицу-ключ."""
        return X @ self.__P_inv
    
    def fit_transform(self, X):
        """Принимает на вход матрицу и преобразует её случайным образом."""
        self.fit(X)
        return self.transform(X)
    
    def __make_matrix(self, X):
        """Создаёт случайную матрицу.
        Пересоздаёт, если матрица необратима."""
        # Задаём случайное состояние, если оно указано при создании объекта
        np.random.seed(self.__rstate)
        
        # Создаём квадратную матрицу заданного размера
        self.__P = np.random.randint(1000, size=(X.shape[1], X.shape[1]))
        
        # С осторожностью находим обратную матрицу
        try:
            self.__P_inv = np.linalg.inv(self.__P)
            
        # Если нет обратной, повторяем
        except np.linalg.LinAlgError:
            self.__make_matrix(X)

---

## 4. Проверка алгоритма
Посмотрим, как работает наш шифровщик. Попутно проверим, как работают модели с его применением. Не будем фиксировать случайное состояние: алгоритм должен срабатывать всегда.

In [25]:
# Создадим объект шифровщика
mc = RandomCipher()

In [26]:
X

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [27]:
# Обучим его на исходных признаках
X_ciphered = mc.fit_transform(X)
X_ciphered

array([[12667214., 26559973., 36743216., 35787317.],
       [ 9710981., 20356369., 28163612., 27426107.],
       [ 5368079., 11251124., 15567086., 15158632.],
       ...,
       [ 8653990., 18149206., 25105976., 24454338.],
       [ 8349615., 17509495., 24220766., 23591143.],
       [10366351., 21737745., 30071074., 29290413.]])

In [28]:
# Убедимся, что качество модели не изменилось
float_equal(Linear_r2(X_ciphered, y).score, r2_initial.score)

True

Качество модели не изменилось. Зато данные перестали быть понятными для человека.

Проверим, как работает расшифровка.

In [29]:
# Расшифруем данные
X_deciphered = mc.inverse_transform(X_ciphered)
X_deciphered

array([[ 1.00000000e+00,  4.10000000e+01,  4.96000000e+04,
         1.00000000e+00],
       [-7.27595761e-12,  4.60000000e+01,  3.80000000e+04,
         1.00000000e+00],
       [-1.81898940e-12,  2.90000000e+01,  2.10000000e+04,
         3.63797881e-12],
       ...,
       [-3.63797881e-12,  2.00000000e+01,  3.39000000e+04,
         2.00000000e+00],
       [ 1.00000000e+00,  2.20000000e+01,  3.27000000e+04,
         3.00000000e+00],
       [ 1.00000000e+00,  2.80000000e+01,  4.06000000e+04,
         1.00000000e+00]])

Признаки выглядят похоже на исходные.

In [30]:
# Сравним исходные и расшифрованные признаки как float
np.all(float_equal(X, X_deciphered))

True

In [31]:
# Проверим качество на расшифрованных данных
float_equal(Linear_r2(X_deciphered, y).score, r2_initial.score)

True

Что бы мы ни делали с данными, регрессия даёт одинаковый результат. А значит, что наш класс работает верно.

---

## 5. Общий вывод
- Мы изучили возможность преобразования данных при помощи обратимой матрицы и привели математическое доказательство того, что **умножение признаков на обратимую матрицу не влияет на качество линейной регрессии**.
- Написали класс-инструмент `RandomCipher` в стиле `sklearn`, который умеет **преобразовывать матрицы случайным образом** и возвращать их в исходный вид, не делясь при этом ключом с пользователем.
- Убедились, что линейная регрессия предсказывает на зашифрованных данных не хуже, чем на исходных.

Теперь мы сможем использовать наш класс `RandomCipher` для того, чтобы легко и быстро скрывать реальные значения признаков и сохранять при этом качество предсказаний линейной регрессии.

---